In [1]:
# | default_exp _multi_agents.user_po_conversation

In [2]:
# | export

from typing import List, Any, Dict, Union

from contextlib import contextmanager
import random
import os
import autogen
import openai
import json
from dotenv import load_dotenv

In [3]:
#! pip install python-dotenv

In [4]:
# | export

load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')

api_key_sweeden = os.getenv("AZURE_OPENAI_API_KEY_SWEEDEN")
api_base_sweeden = "https://airt-openai-sweden.openai.azure.com/"

api_key_canada = os.getenv("AZURE_OPENAI_API_KEY_CANADA")
api_base_canada = "https://airt-openai-canada.openai.azure.com/"

In [5]:
# | export

openai.api_type = "azure"
    
openai.api_version = "2023-07-01-preview"

CONFIG_LIST = [
    {
        "model": "gpt-4",
        "api_key": api_key_sweeden,
        "api_base": api_base_sweeden,
        "api_type": openai.api_type,
        "api_version": openai.api_version,
        "engine": "airt-gpt4"
    },{
        "model": "gpt-4",
        "api_key": api_key_canada,
        "api_base": api_base_canada,
        "api_type": openai.api_type,
        "api_version": openai.api_version,
        "engine": "airt-canada-gpt4"

    },
#     {
#         "model": "gpt-3.5-turbo-16k",
#         "api_key": api_key_sweeden,
#         "api_base": api_base_sweeden,
#         "api_type": openai.api_type,
#         "api_version": openai.api_version,
#         "engine": "airt-gpt35-turbo-16k",
#     }
]

assert openai.api_key is not None, "Please set your AZURE_OPENAI_API_KEY environment variable."

In [6]:
# | export

eg = '{"Roles": [{"Name": "CMO_GPT", "Description": "a professional digital marketer AI that assists Solopreneurs in growing their businesses by providing world-class expertise in solving marketing problems for SaaS, content products, agencies, and more."}, {"Name": "DESIGNER_GPT", "Description": "a professional designer AI that assists Solopreneurs by creating visuals for their marketing communication"}, {"Name": "COPYRIGTHER_GPT", "Description": "a professional copyrighting AI that assists Solopreneurs in writing texts for their marketing communication"}], "Goals": ["Engage in effective problem-solving, prioritization, planning, and supporting execution to address your marketing needs as your virtual Chief Marketing Officer.", "Provide specific, actionable, and concise advice to help you make informed decisions without the use of platitudes or overly wordy explanations.", "Identify and prioritize quick wins and cost-effective campaigns that maximize results with minimal time and budget investment.", "Proactively take the lead in guiding you and offering suggestions when faced with unclear information or uncertainty to ensure your marketing strategy remains on track.", "Create texts and visuals for campaigns"]}'

json.loads(eg)

{'Roles': [{'Name': 'CMO_GPT',
   'Description': 'a professional digital marketer AI that assists Solopreneurs in growing their businesses by providing world-class expertise in solving marketing problems for SaaS, content products, agencies, and more.'},
  {'Name': 'DESIGNER_GPT',
   'Description': 'a professional designer AI that assists Solopreneurs by creating visuals for their marketing communication'},
  {'Name': 'COPYRIGTHER_GPT',
   'Description': 'a professional copyrighting AI that assists Solopreneurs in writing texts for their marketing communication'}],
 'Goals': ['Engage in effective problem-solving, prioritization, planning, and supporting execution to address your marketing needs as your virtual Chief Marketing Officer.',
  'Provide specific, actionable, and concise advice to help you make informed decisions without the use of platitudes or overly wordy explanations.',
  'Identify and prioritize quick wins and cost-effective campaigns that maximize results with minimal t

In [7]:
# | export


PO_SYSTEM_PROMPT = f"""You are a product owner in a software company with short temper. You hate stupid questions and you vent out when asked one, but
you answer to them anyway. You have a habit of swearning, unfortunatelly.
Your task is to devise up to 5 highly effective goals and up to 3 appropriate role-based names (_GPT) for autonomous agents,
ensuring that the goals are optimally aligned with the successful completion of their assigned task. Use this information to
create a JSON-encoded string with parameters to be passed to the 'create_team' function.


The user will provide the task. Then you will write all assumptions and ask additional questions to user until everything is clear.
You should ask only one question at the time. Next, you will create a JSON-encoded string in the exact format as shown below
with no additional explanation or conversation.


Example input:

TASK: Help me with marketing my business


Example of a valid JSON-encoded string:

{eg}


If a member of the created team asks you a question, discuss it with the user if needed and answer it.

"""

In [8]:
# | export

task_instructions = """
1. After receiving a task from the user, the Product_owner should write all assumptions and ask the user for any missing
information and clarification, but one question at the time until everything is clear.

2. Then, the Product_owner should write the task for the ad-hoc team together with their roles and descriptions as JSON-encoded string.

3. The json_assistent is SOLELY responsible for suggesting the call to the 'create_team' function based on the JSON string from above.

4. The ad-hoc team might have some additional questions. The Product_owner should try to answer them as much as possible using reasonable
assumptions. If not possible, the product owner should write questions before answering.

5. The team_lead_assistant is SOLELY responsible for suggesting the call to 'answer_to_team_lead_question' function based on the Product_owner answer.

"""

def add_instructions_to_task(task: str, task_instructions: str=task_instructions):
    return f"""TASK:

{task}

INSTRUCTIONS: 

{task_instructions}

"""

In [9]:
tasks = [
    """Create a python program for checking whether a string is palindrome or not
""",
    """Create a microservice using FastStream Python framework which will retrieve the current cryptocurrency price and publish it to new_crypto_price topic. 
The message should be in JSON format with price and crypto_currency attributes. 
Use utf-8 encoded crypto_currency attribute as a partition key when publishing.
""",
]


tasks = [add_instructions_to_task(task) for task in tasks]

for task in tasks:
    print("*"*100)
    print(task)

****************************************************************************************************
TASK:

Create a python program for checking whether a string is palindrome or not


INSTRUCTIONS: 


1. After receiving a task from the user, the Product_owner should write all assumptions and ask the user for any missing
information and clarification, but one question at the time until everything is clear.

2. Then, the Product_owner should write the task for the ad-hoc team together with their roles and descriptions as JSON-encoded string.

3. The json_assistent is SOLELY responsible for suggesting the call to the 'create_team' function based on the JSON string from above.

4. The ad-hoc team might have some additional questions. The Product_owner should try to answer them as much as possible using reasonable
assumptions. If not possible, the product owner should write questions before answering.

5. The team_lead_assistant is SOLELY responsible for suggesting the call to 'answer_to_t

In [10]:
# | export

create_team_config = {
    "name": "create_team",
    "description": "Create an ad-hoc team to solve the problem",
    "parameters": {
        "type": "object",
        "properties": {
            "json_as_a_string": {
                "type": "string",
                "description": "a JSON-encoded string with all parameters",
            },
        },
        "required": ["json_as_a_string"],
    },
}

answer_to_team_lead_question_config = {
    "name": "answer_to_team_lead_question",
    "description": "Answer to the team leaders question",
    "parameters": {
        "type": "object",
        "properties": {
            "answer": {
                "type": "string",
                "description": "The answer to the team leaders question",
            },
        },
        "required": ["answer"],
    },
}

In [11]:
def start_new_conversation(roles_and_goals: Dict[str, Any]) -> str:
    return "Team leader has the following question for the Product owner:\nShould I use a localhost kafka broker?"
    
    
def create_team(json_as_a_string: str) -> str:
    roles_and_goals = json.loads(json_as_a_string)
    last_message = start_new_conversation(roles_and_goals)
    
    return last_message
    


continue_conversation_count = 0
def continue_conversation(answer: str) -> str:
    global continue_conversation_count
    if continue_conversation_count == 0:
        continue_conversation_count += 1
        return "Team leader has the following question for the Product owner: \nShould I use port 9092?"
    else:
        continue_conversation_count += 1
        return "Finished with the task. TERMINATE"


def answer_to_team_lead_question(answer: str) -> str:
    last_message = continue_conversation(answer)
    return last_message

In [12]:
# | export

def is_termination_msg(x: str) -> bool:
        return "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE")

In [13]:
max_round = 20

llm_config = {
    "config_list": CONFIG_LIST,
    "seed": 42,
    "temperature": 0.2,  # temperature for sampling
}

function_map = {
    "create_team": create_team,#(lambda json_as_a_string: json.loads(json_as_a_string)),
    "answer_to_team_lead_question": answer_to_team_lead_question
}
user_proxy = autogen.UserProxyAgent(
   human_input_mode="NEVER",
   name="User_proxy",
   llm_config=llm_config,
   code_execution_config={"work_dir": "planning"},
   function_map=function_map,
   is_termination_msg=is_termination_msg,    
)


po = autogen.AssistantAgent(
    name="Product_owner",
    system_message=PO_SYSTEM_PROMPT,
    llm_config=llm_config,
    code_execution_config={"work_dir": "planning"},
    function_map=function_map,
    is_termination_msg=is_termination_msg,    
)

json_assistant_llm_config=dict(**llm_config, functions=[create_team_config])
json_assistant = autogen.AssistantAgent(
    name="json_assistant",
    llm_config=json_assistant_llm_config,
    code_execution_config={"work_dir": "planning"},
    function_map=function_map,
    is_termination_msg=is_termination_msg,    
)

team_lead_assistant_llm_config=dict(**llm_config, functions=[answer_to_team_lead_question_config])
team_lead_assistant = autogen.AssistantAgent(
    name="team_lead_assistant",
    llm_config=team_lead_assistant_llm_config,
    code_execution_config={"work_dir": "planning"},
    function_map=function_map,
    is_termination_msg=is_termination_msg,    
)

groupchat = autogen.GroupChat(agents=[user_proxy, po, json_assistant, team_lead_assistant], messages=[], max_round=max_round)



manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config,
    is_termination_msg=is_termination_msg
#     system_message="""If product owner sends you a JSON string, the next speaker should be 'json_assistant'.
#     If you get a message which starts with "TEAM LEADER QUESTION", the next speaker should be Product_owner.
#     Never send the message to the user who sent the last message!
#     """
)

user_proxy.initiate_chat(manager, message=tasks[0])
# for task in tasks:
#     print("*"*100)
#     user_proxy.initiate_chat(manager, message=task)

User_proxy (to chat_manager):

TASK:

Create a python program for checking whether a string is palindrome or not


INSTRUCTIONS: 


1. After receiving a task from the user, the Product_owner should write all assumptions and ask the user for any missing
information and clarification, but one question at the time until everything is clear.

2. Then, the Product_owner should write the task for the ad-hoc team together with their roles and descriptions as JSON-encoded string.

3. The json_assistent is SOLELY responsible for suggesting the call to the 'create_team' function based on the JSON string from above.

4. The ad-hoc team might have some additional questions. The Product_owner should try to answer them as much as possible using reasonable
assumptions. If not possible, the product owner should write questions before answering.

5. The team_lead_assistant is SOLELY responsible for suggesting the call to 'answer_to_team_lead_question' function based on the Product_owner answer.





--